In [ ]:
%pylab notebook
from __future__ import print_function 
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 720)
pd.set_option('display.max_columns', 30)

In [ ]:
%cd /Users/brodzik/2018/POLAR2018/
%ls

# Read spreadsheets and clean up headers

In [ ]:
station = "Khunjerab"
file = 'TbQnew2018.working.xlsx'
station_elevation_m = 4440
df = pd.read_excel(open(file,'rb'), sheetname='AMSREV1.3_Near Kunjerab2003')
melt_onset = '2003-04-12'

# Make a new column with just the day fraction part of DOY
df['day_fraction'] = [i - int(i) for i in df.DOY.values]

# And then add the day fraction to the Date value
df['Date'] = [pd.to_datetime(i) + pd.DateOffset(days=j) for i, j in zip(df.Date, df.day_fraction)]

# And drop the day_fraction
df.drop(['day_fraction'], axis=1, inplace=True)

# Move the df index to the Date column
df = df.set_index('Date')
df = df[['TB: 36 GHZ V (K) 220,44', 'DAV: 36 GHZ V (K)220,44']]
df.columns = ['TB_36V_K', 'DAV_36V_K']
df

# Read the original data from Joan for station min/max temperatures

In [ ]:
file = '/Users/brodzik/2017/AGU_Hunza_melt_onset/Khunjerabnear_uib50-263.working.xlsx'
t_df = pd.read_excel(open(file,'rb'), sheetname='pixelUIB50-263glnearKh')

# Drop the 1st 2 rows, we don't need them
t_df = t_df.drop([0,1])

# Move the contents of data row 2 into the column headers
# and then drop row 2
t_df.columns = t_df.loc[2].values
t_df = t_df.drop([2])

# Move the df index to the Date_time_utc column
#df = df.set_index('Date_time_utc')

# Just save the station temperatures
t_df = t_df[['doy', 'T_min_C', 'T_max_C']]

# Move the df index to the Date_time_utc column
t_df = t_df.set_index('doy')
t_df.T_max_C


# Read the Hunza daily discharge data

In [ ]:
dischargeFile = '/Users/brodzik/projects/CHARIS/streamflow/Pakistan/Hunza/hunza_at_dainyor_bridge_daily_timeseries_2003.txt'
hunza = pd.read_csv(dischargeFile, header=None, sep='\s+',
                    names=['year', 'month', 'day', 'doy', 'Q', 'dummy'])

hunza['Date'] = pd.to_datetime(hunza.year*10000 + hunza.month*100 + hunza.day,format='%Y%m%d')
hunza.set_index('Date', drop=True, inplace=True)
hunza.drop(['year','month','day', 'doy', 'dummy'], axis=1, inplace=True)
hunza

In [ ]:
melt0 = ['2003-04-12', '2003-04-13']
melt1 = ['2003-05-07', '2003-05-09']
melt2 = ['2003-05-21', '2003-10-05']

In [ ]:
do_annotate = True
fig = plt.figure(figsize=(11,8.5))
gs = gridspec.GridSpec(3, 1, height_ratios=[3,3,1])

ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])
ax2 = plt.subplot(gs[2])
line_36v = df["TB_36V_K"].plot(ax=ax0, color='mediumslateblue', 
                               label='$T_B$ 36V',
                               marker='.', linestyle=':')
ax0.set_title('Glacier near %s (%d m): $T_B$ (2003)' % (
    station, station_elevation_m))
ax0.set_ylim([165., 300.])
ax0.set_xlabel("Date")
ax0.set_ylabel("$T_B$ ($K$)")

# Add a line for TB melt threshold
ax0.axhline(y=254., color='mediumslateblue', 
            linestyle="-", 
            label="$T_B$ threshold")

ax0.grid(linestyle=':', color='lightgray')
    
ax0.legend(loc='upper right')

if do_annotate:
    ax0.annotate('',
                 xy=(pd.to_datetime(melt0[0]), 170.), 
                 xytext=(pd.to_datetime(melt0[0]), 140.),
                 arrowprops=dict(facecolor='orangered', shrink=0.05))
    ax0.annotate('',
                 xy=(pd.to_datetime(melt1[0]), 170.), 
                 xytext=(pd.to_datetime(melt1[0]), 140.),
                 arrowprops=dict(facecolor='orangered', shrink=0.05))


# Work on middle plot
line_dav_36v = df["DAV_36V_K"].plot(ax=ax1, 
                                    color='gray',
                                    label='|$DAV_{36V}$|')


ax1.set_title('Glacier near %s (%d m): DAV and Station Temperatures' % (
    station, station_elevation_m))
ax1.set_ylim([-30., 110.])
ax1.set_xlabel("Date")
ax1.set_ylabel("Degrees")

line_tmax = t_df["T_max_C"].plot(ax=ax1,
                                 color='green', 
                                 label="T_max")
line_tmin = t_df["T_min_C"].plot(ax=ax1,
                                 color='lawngreen', 
                                 label="T_min")

ax1.grid(linestyle=':', color='lightgray') 

# Add a line for DAV melt threshold
ax1.axhline(y=40., color='gray',
            linestyle="-", 
            label="$DAV$ threshold")

# Add a line for freezing surface temperature
ax1.axhline(color='g', linestyle=":", label="$T = 0^o C$")


if do_annotate:
    ax1.annotate('Early Melt Events', 
                 xy=(pd.to_datetime(melt0[0]), 100.), 
                 xytext=(pd.to_datetime(melt0[0]), 130.),
                 arrowprops=dict(facecolor='orangered', shrink=0.05),
                 fontsize=16,
                 color='orangered',
                 horizontalalignment='center')
    ax1.annotate('', 
                 xy=(pd.to_datetime(melt1[0]), 100.), 
                 xytext=(pd.to_datetime(melt1[0]), 130.),
                 arrowprops=dict(facecolor='orangered', shrink=0.05),
                 fontsize=16,
                 color='orangered',
                 horizontalalignment='center')



# Plot likely melt onset date
#line_v = ax1.axvline(pd.to_datetime(melt_onset), 
#                      label='$T_{max} > 0^o C$',
#                      color='k', linestyle=':', linewidth=3.0)

ax1.legend(loc="upper right")

# Shade grey area for intermittent and ongoing melt periods
if do_annotate:
    for ax in [ax0, ax1]:
        ylim = ax.get_ylim()
        for melt in [melt0, melt1, melt2]:
            ax.fill_between([pd.to_datetime(melt[0]), pd.to_datetime(melt[1])],
                            [ylim[0], ylim[0]], 
                            [ylim[1], ylim[1]], 
                            facecolor='lightgray', alpha=0.5)
        
    ax1.text(pd.to_datetime('2003-07-28'),
             95.,
             "High DAV (Melt Season)",
             fontsize=14,
             color='orangered',
             horizontalalignment='center')
    ax1.annotate(s='', 
                 xy=(pd.to_datetime(melt2[0]), 90.), 
                 xytext=(pd.to_datetime(melt2[1]), 90.), 
                 arrowprops=dict(arrowstyle='<->',
                                 color='orangered',
                                 linewidth=2))
 
# Third plot is Hunza discharge
line_Q = hunza["Q"].plot(ax=ax2, color='k', label='Hunza at Dainyor Bridge')
ax2.set_title('Measured discharge, Hunza at Dainyor Bridge (1370 m), 2003')
ax2.set_ylabel('Q ($m^3 s^{-1}$)')
ax2.grid(linestyle=':', color='lightgray')
              
fig.tight_layout()
if do_annotate:
    outfile = "/Users/brodzik/2018/POLAR2018/%s_36V_DAV_2003_tseries.annotation.png" % station
else:
    outfile = "/Users/brodzik/2018/POLAR2018/%s_36V_DAV_2003_tseries.png" % station
#outfile = "/Users/brodzik/2018/POLAR2018/%s_36V_DAV_2003_tseries.annotation.pdf" % station
plt.savefig(outfile)
print("Image saved to %s" % outfile)

In [ ]:
ax1

In [ ]:
ax1.get_ylim()
